<a href="https://colab.research.google.com/github/pengfei123xiao/ML_Basic/blob/master/models/Ch05-DecisionTree/RF_GBDT_XGBoost_LightGBM/GBDT_Key_Points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GBDT Key Points
GBDT也是集成学习Boosting家族的成员，但是却和传统的Adaboost有很大的不同。回顾下Adaboost，是利用前一轮迭代弱学习器的误差分布来更新训练集的权重，这样一轮轮的迭代下去。

GBDT也是迭代，使用了前向分布算法，但是**弱学习器只能使用CART回归树模型**，同时迭代思路和Adaboost也有所不同。

在GBDT的迭代中，假设我们前一轮迭代得到的强学习器是$f_{t-1}(x)$, 损失函数是$L(y, f_{t−1}(x))$, 我们本轮迭代的目标是找到一个CART回归树模型的弱学习器$h_t(x)$，让**本轮损失的损失** $L(y,f_t(x)=L(y, f_{t-1}(x)+h_t(x))$最小。

GBDT的思想可以用一个通俗的例子解释，假如有个人30岁，我们首先用20岁去拟合，发现损失有10岁，这时我们用6岁去拟合剩下的损失，发现差距还有4岁，第三轮我们用3岁拟合剩下的差距，差距就只有一岁了。如果我们的迭代轮数还没有完，可以继续迭代下面，每一轮迭代，拟合的岁数误差都会减小。


### 1. 前向分布算法
加法模型，从前向后，每步只学习一个基函数及其系数，逐步逼近目标。

**1.1** 加载模型
   
   $f(x)=\displaystyle\sum^M_{m=1}\beta_mb(x;\gamma_m )$ (基学习器的一种线性模型)
   
   其中，$b(x;\gamma_m )$ 为基函数，$\gamma_m$作为基函数的参数，$\beta_m$作为基函数的系数。
   
   在给定训练数据的损失函数$L(y,f(x))$的条件下，学习加法模型成为损失函数极小化问题：
   
   $min_{\beta_m,\gamma_m}\displaystyle\sum^N_{i=1}L(y_i,\displaystyle\sum^M_{m=1}\beta_mb(x_i;\gamma_m))$(同时求解那么多参数)
   
   前向分步算法求解这一优化问题的思路：因为学习的是加法模型，如果能够从前向后，**每一步只学习一个基函数及其系数，逐步去逼近上述的目标函数式，就可简化优化的复杂度，**每一步只需优化如下损失函数：
   
   $min_{\beta_m,\gamma_m}\displaystyle\sum^N_{i=1}L(y_i,\beta_mb(x;\gamma))$(每步学习一个基函数和系数)
   
**前向分步算法流程：**

---------------------------------------

输入：训练数据集$T=({ (x_1,y_1),(x_2,y_2),...(x_N,y_N) })$；损失函数$L(y,f(x))$；基函数集${ b(x,\gamma) }$；

输出：加法模型f(x)

(1) 初始化 $f_0(x)=0$

(2) 对$m=1,2,...,M$

(a) 极小化损失函数

$(\beta,\gamma)=argmin_{\beta,\gamma}\displaystyle\sum^N_{i=1}L(y_i,f_{m-1}(x_i)+\beta b(x_i;\gamma))$

得到参数 $\beta_m,\gamma_m$

(b)更新

$f_m(x)=f_{m-1}+\beta_m b(x;\gamma_m)$

(3)得到加法模型

$f(x)=f_M(x)=\displaystyle\sum^M_{m=1}\beta_m b(x;\beta_m)$

---------------------------------------

可见，前向分步算法将同时求解从m=1到M所有参数 $\beta_m,\gamma_m$ 的优化问题简化成逐步求解各个 $\beta_m,\gamma_m$的优化问题了

### 2. 负梯度拟合
**2.1 提升树算法**

了解完前向分步算法，再来看看什么是提升树算法。

提升方法实际采用*加法模型与前向分步算法*，以**决策树作为基函数**的提升方法称为提升树。注意，这里的**决策树为CART回归树**，不是分类树。当问题是分类问题时，采用的决策树模型为分类回归树。为什么要采用决策树作为基函数呢？它有以下优缺点：

**优点:**
- 可解释性强
- 可处理混合类型特征
- 具有伸缩不变性（不用归一化特征）
- 有特征组合的作用
- 可自然处理缺失值
- 对异常点鲁棒
- 有特征选择作用
- 可扩展性强，容易并行

**缺点:**
- 缺乏平滑性（回归预测时输出值只能是输出有限的若干种数值）
-不适合处理高维稀疏数据
-由于树的线性组合可以很好地拟合训练数据，即使数据中的输入与输出之间的关系很复杂也是如此。

提升树模型可表示为：

$f_M(x)=\displaystyle\sum^M_{m=1}T(x;\theta_m)$

其中， $T(x;\theta_m)$ 表示决策树;$\theta$ 为决策树的参数;M为树的个数;M为树的个数。

针对不同的问题，提升树算法的形式有所不同，其主要区别在于使用的**损失函数不同**。而损失函数的不同，决策树要拟合的值也会不同。就一般而言，对于回归问题的提升树算法来说，若损失函数是平方损失函数，每一步只需简单拟合当前模型的残差。

下面看一下在回归问题下，损失函数为平方损失函数的算法流程：

---------------------------------------

输入：训练数据集$T=({ (x_1,y_1),(x_2,y_2),...(x_N,y_N) }),x_i \in \chi \subseteq R^n,y_i \in \gamma \subseteq R$ ；

输出：提升树 $f_M(x)$

(1)初始化 $f_0(x)=0$

(2)对m=1,2,...,M

(a)计算残差

$r_{mi}=y_i-f_{m-1}(x_i),i=1,2,...N$

(b)拟合残差 $r_{mi}$ 学习一个回归树，得到 $T(x;\theta_m)$

(c)更新$f_m(x)=f_{m-1}(x)+T(x;\theta_m)$

(3)得到回归问题提升树

$f_M(x)=\displaystyle\sum^M_{m=1}T(x;\theta_m)$

---------------------------------------

**2.2 梯度提升**

提升树用加法模型与前向分布算法实现学习的优化过程。当损失函数为平方损失和指数损失函数时，每一步优化是很简单的。但对于一般损失函数而言，往往每一步都不那么容易。对于这问题，Freidman提出了梯度提升算法。这是利用最速下降法的近似方法，其关键是利用损失函数的负梯度在当前模型的值：

$-[\frac{\partial L(y,f(x_i))}{\partial f(x_i)}]_{f(x)}=f_{m-1}(x)$

作为回归问题在当前模型的残差的近似值，拟合一个回归树。

为什么要拟合负梯度呢？这就涉及到泰勒公式和梯度下降法了。

泰勒公式的形式是这样的：

定义：泰勒公式是一个用函数在某点的信息描述其附近取值的公式。

基本形式： $f(x)=\displaystyle\sum^\infty_{n=1}\frac{f^{(n)}(x_0)}{n!}(x-x_0)^n$

一阶泰勒展开： $f(x)\approx f(x_0)+f'(x_0)(x-x_0)$

梯度下降法

在机器学习任务中，需要最小化损失函数 $L(\theta)$ ，其中$\theta$ 是要求解的模型参数。梯度下降法常用来求解这种无约束最优化问题，它是一种迭代方法：选择初值 $\theta^0$ ，不断迭代更新 $\theta$ 的值，进行损失函数极小化。

迭代公式： $\theta^t=\theta^{t-1}+\Delta\theta$

将 $L(\theta^t)$ 在 $\theta^{t-1}$ 处进行一阶泰勒展开： $L(\theta^t)=L(\theta^{t-1}+\Delta\theta))\approx L(\theta^{t-1})+L'(\theta^{t-1})\Delta\theta$

要使得 $L(\theta^t)<L(\theta^{t-1})$ ，可取： $\Delta\theta=-\alpha L'(\theta^{t-1})$ ，

则： $\theta=\theta^{t-1}-\alpha L'(\theta^{t-1})$ ，这里 $\alpha$ 是步长，一般直接赋值一个小的数。

相对的，在函数空间里，有 $f^t(x)=f^{t-1}(x)+f_t(x)$

此处把 $L(f^t(x))$ 看成提升树算法的第t步损失函数的值， $L(f^{t-1}(x))$ 为第t-1步损失函数值，要使 $L(f^t(x))<L(f^{t-1}(x))$ ，

则需要 $f_t(x)=-\alpha g_t(x)$ ，此处 $-g_t(x)$ 为**当前模型的负梯度值，** 即第t步的回归树需要拟合的值。

至于GBDT的具体算法流程，在后续回归与分类问题会分开说明。

-----------------------
用损失函数的负梯度来拟合本轮损失的近似值，进而拟合一个CART回归树。通过损失函数的负梯度来拟合，找到了一种通用的拟合损失误差的办法，这样无论是分类问题还是回归问题，我们通过其**损失函数的负梯度的拟合**，就可以用GBDT来解决我们的分类回归问题。区别仅仅在于损失函数不同导致的负梯度不同而已。

第$t$轮的第$i$个样本的损失函数的负梯度表示为:
$r_{ti}=-[\frac{\partial L(y, f(x_i))}{\partial f(x_i)}]_{f(x)=f_{t-1}(x)}$

利用$(x_i,r_{ti})(i=1,2,..m)$, 我们可以拟合一颗CART回归树，得到了第$t$颗回归树，其对应的叶节点区域$R_{t_j,j}=1,2,...,J$。其中$J$为叶子节点的个数。

针对每一个叶子节点里的样本，我们求出使损失函数最小，也就是拟合叶子节点最好的的输出值$c_{t_j}$如下：

### 3. 损失函数
在GBDT算法中，损失函数的选择十分重要。针对不同的问题，损失函数有不同的选择。

1.对于分类算法，其损失函数一般由对数损失函数和指数损失函数两种。

(1)指数损失函数表达式：

$L(y,f(x))=e^{(-yf(x))}$

(2)对数损失函数可分为二分类和多分类两种。

2.对于回归算法，常用损失函数有如下4种。

(1)平方损失函数：

$L(y,f(x))=(y-f(x))^2$

(2)绝对损失函数：

$L(y,f(x))=|y-f(x)|$

对应负梯度误差为：

$L(y,f(x))=sign(y_i-f(x_i))$

(3)Huber损失，它是均方差和绝对损失的折中产物，对于远离中心的异常点，采用绝对损失误差，而对于靠近中心的点则采用平方损失。这个界限一般用分位数点度量。损失函数如下：

$L(y,f(x))=\left\{\begin{matrix} \frac{1}{2}(y-f(x))^2&  |y-f(x)| \leqslant\delta&  \\ \delta(|y-f(x)|-\frac{\delta}{2})&  |y-f(x)|>\delta& \end{matrix}\right.$

对应的负梯度误差为：

$r(y_i,f(x_i))=\left\{\begin{matrix}y_i-f(x_i)&  |y_i-f(x)|\leqslant\delta&  \\ \delta sign(y_i-f(_i))&  |y_i-f(x_i)|>\delta& \end{matrix}\right.$


（4）分位数损失。它对应的是分位数回归的损失函数，表达式为：

$L(y,f(x))=\displaystyle\sum_{y \geqslant f(x)} \theta|y-f(x)|+\displaystyle\sum_{y < f(x)} (1-\theta)|y-f(x)|$

其中 $\theta$ 为分位数，需要我们在回归之前指定。对应的负梯度误差为：

$r(y_i,f(x_i))\left\{\begin{matrix}\theta&  y_i \geqslant f(x_i)&  \\ \theta-1 &  y_i \geqslant f(x_i)& \end{matrix}\right.$

对于Huber损失和分位数损失，主要用于健壮回归，也就是减少异常点对损失函数的影响。


### 4. 回归
梯度提升算法（回归问题）流程：

---------------------------------------

输入：训练数据集$T=({ (x_1,y_1),(x_2,y_2),...(x_N,y_N) }),x_i \in \chi \subseteq R^n,y_i \in \gamma \subseteq R$损失函数$L(y,f(x))$；

输出：回归树 $\tilde f(x)$

(1)初始化

$f_0=argmin_c \displaystyle \sum^N_{i=1}L(y_i,c)$ 注：估计使损失函数极小化的常数值，它是只有一个根结点的树

(2)对m=1,2,...,M

(a)对i=1,2,...N，计算

$r_{mi}=-[\frac{\partial L(y,f(x_i))}{\partial f(x_i)}]_{f(x)}=f_{m-1}(x)$ 注： 计算损失函数在当前模型的值，作为残差的估计

(b)对 $r_{mi}$ 拟合一个回归树，得到第m棵树的叶结点区域 $R_{mj}$ ,j=1,2,...,J

(c)对j=1,2,...,J，计算

$c_{mj}=argmin_c \displaystyle \sum _{x_j \in R_{mj}} L(y_i,f_{m-1}(x_i)+c)$ 注：在损失函数极小化条件下，估计出相应叶结点区域的值

(d)更新

$f_m(x)=f_{m-1}(x)+ \displaystyle \sum^J_{j=1}c_{mj}I(x \in R_{mj})$

(3)得到回归树

$\tilde f(x)=fM(x)=\displaystyle\sum^M_{m=1} \displaystyle\sum^J_{j=1} c_{mj}I(x \in R_{mj})$

### 5. 二分类，多分类

这里看看GBDT分类算法，GBDT的分类算法从思想上和GBDT的回归算法没有区别，但是由于样本输出不是连续的值，而是离散的类别，导致我们无法直接从输出类别去拟合输出类别的误差。

为了解决这个问题，主要有两个方法，一个是用指数损失函数，此时GBDT退化为Adaboost算法。另一种方法用类似逻辑回归的对数似然函数的方法。也就是说，我们用的是类别的预测概率值和真实概率值的差来拟合损失。此处仅讨论用对数似然函数的GBDT分类。对于对数似然损失函数，我们有又有二元分类和的多元分类的区别。

1.二分类GBDT算法

对于二分类GBDT，如果用类似逻辑回归的对数似然损失函数，则损失函数为：

$L(y,f(x))=log(1+exp(-yf(x)))$

其中 $y \in ${-1,1} 。此时的负梯度误差为：

$r_{ti}=-\frac{\partial L(y_i,f_{t-1})}{\partial f_{t-1}(x_i)}=\frac{y_i}{1+exp(y_i f_{t-1}(x_i))},i=1,2,3...,m$

对于生成的决策树，我们各个叶子节点的最佳负梯度拟合值为

$c_{tj}=argmin_c \displaystyle \sum_{x_i \in R_{tj}} log(1+exp(-y_i(f_{t-1}(x_i)+c)))$

由于上式比较难优化，我们一般使用近似值代替

$c_{tj}=\frac{\displaystyle \sum_{x_i \in R_{tj}} r_{tj}}{\displaystyle \sum_{x_i \in R_{tj}}|r_{tj}|(1-|r_{tj}|)}$

除了负梯度计算和叶子节点的最佳负梯度拟合的线性搜索，二分类GBDT与GBDT回归算法过程相同。

2.多分类GBDT算法

多分类GBDT比二分类GBDT复杂些，对应的是多元逻辑回归和二元逻辑回归的复杂度差别。假设类别数为K，则此时我们的对数似然损失函数为：

$L(y,f(x))-\displaystyle \sum^K_{k=1} y_k log(p_k(x))$

其中如果样本输出类别为k，则 $y_k$ =1.第k类的概率 $p_k(x)$ 的表达式为：

$p_k(x)=\frac{exp(f_k(x))}{\sum^K_{l=1}}exp(f_l(x))$

集合上两式，我们可以计算出第t轮的第i个样本对应类别l的负梯度误差为：

$t_{til}=-[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}]_{f_k(x)=f_{l,t-1}(x)}=y_{il}-p_{l,t-1}(x_i)$

观察上式可以看出，其实这里的误差就是样本i对应类别l的真实概率和t-1轮预测概率的差值。

对于生成的决策树，我们各个叶子节点的最佳负梯度拟合值为：

$c_{tjl}=argmin_{c_{jl}} \displaystyle\sum^m_{i=1} \displaystyle\sum^K_{k=1} L(y_k,f_{t-1,l}(x)+\displaystyle\sum^J_{j=1} c_{jl}I(x_i \in R_{tj}))$

由于上式比较难优化，我们一般使用近似值代替

$c_{tjl}=\frac{K-1}{K}\frac{\sum_{x_i \in R_{tjl}} r_{til}}{\sum_{x_i \in R_{tjl}}|r_{til}|(1-|r_{til}|)}$

除了负梯度计算和叶子节点的最佳负梯度拟合的线性搜索，多分类GBDT与二分类GBDT以及GBDT回归算法过程相同。

### 6. 正则化
对GBDT进行正则化来防止过拟合，主要有三种形式。

>1.给每棵数的输出结果乘上一个步长$\alpha$（learning rate）。

对于前面的弱学习器的迭代：

$f_m(x)=f_{m-1}(x)+T(x;\gamma_m)$

加上正则化项，则有

$f_m(x)=f_{m-1}(x)+\alpha T(x;\gamma_m)$

此处，$\alpha$的取值范围为(0,1]。对于同样的训练集学习效果，较小的$\alpha$意味着需要更多的弱学习器的迭代次数。通常我们用步长和迭代最大次数一起决定算法的拟合效果。

>2.第二种正则化的方式就是通过子采样比例(subsample)。取值范围为(0,1]。

GBDT这里的做法是在每一轮建树时，样本是从原始训练集中采用无放回随机抽样的方式产生，与随机森立的有放回抽样产生采样集的方式不同。若取值为1，则采用全部样本进行训练，若取值小于1，则不选取全部样本进行训练。选择小于1的比例可以减少方差，防止过拟合，但可能会增加样本拟合的偏差。取值要适中，推荐[0.5,0.8]。

3.第三种是对弱学习器即CART回归树进行正则化剪枝。（如控制树的最大深度、节点的最少样本数、最大叶子节点数、节点分支的最小样本数等

 

>3. 对弱学习器即CART回归树进行正则化剪枝。

### 7. 优缺点

**优点**

1) 可以灵活处理各种类型的数据，包括连续值和离散值。

2) 在相对少的调参时间情况下，预测的准确率也可以比较高。这个是相对SVM来说的。

3）使用一些健壮的损失函数，对异常值的鲁棒性非常强。比如 Huber损失函数和Quantile损失函数。

**缺点**

1) 由于弱学习器之间存在依赖关系，难以并行训练数据。不过可以通过自采样的SGBT来达到部分并行


### 8. sklearn参数


In [0]:
class sklearn.ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.1,
                                                  n_estimators=100, subsample=1.0, 
                                                  criterion='friedman_mse', 
                                                  min_samples_split=2, min_samples_leaf=1,
                                                  min_weight_fraction_leaf=0.0, 
                                                  max_depth=3, min_impurity_decrease=0.0,
                                                  min_impurity_split=None, init=None, 
                                                  random_state=None, max_features=None, 
                                                  verbose=0, max_leaf_nodes=None, 
                                                  warm_start=False, presort='auto', 
                                                  validation_fraction=0.1, 
                                                  n_iter_no_change=None, tol=0.0001)


`loss`：损失函数

`learning_rate`：学习率

`n_estimators`：基础分类器数量

`subsample`：训练基础分类器样本比例

`criterion`：划分衡量指标

`min_samples_split`：决策树叶结点继续分裂最小样本数量

`min_samples_leaf`：决策树叶结点最小样本数量

`min_weight_fraction_leaf`：决策树叶结点最小加权样本数量

`max_depth`：决策树最大深度

`min_impurity_decrease`：决策树叶结点最小衡量指标提升

`init`：初始化方法

`random_state`：随机种子

`max_features`：搜索划分时考虑的特征数量

`verbose`：控制输出

`max_leaf_nodes`：决策树最大叶结点数量

`warm_start`：是否使用之前的输出

`presort`：是否尝试预先排序数据

`validation_fraction`：验证集比例，控制算法是否提前结束

`n_iter_no_change`：控制算法是否提前结束

`tol`：控制算法是否提前结束

### 9. 应用场景
1. Facebook使用其来自动发现有效的特征、特征组合，来作为LR模型中的特征，以提高 CTR预估（Click-Through Rate Prediction）的准确性，详见Practical Lessons from Predicting Clicks on Ads at Facebook。

2. GBDT在淘宝的搜索及预测业务上也发挥了重要作用。淘宝搜索/推荐系统背后深度强化学习与自适应在线学习的实践之路

3. 美团也有相关的实践：即时配送的ETA问题之亿级样本特征构造实践

**参考**：

西瓜书
          
cs229吴恩达机器学习课程
           
李航统计学习
           
https://ask.hellobi.com/blog/guodongwei1991/10501

公式推导参考：http://t.cn/EJ4F9Q0